In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.5MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 38.8MB/s 
     |████████████████████████████████| 901kB 48.5MB/s 
     |████████████████████████████████| 1.2MB 8.7MB/s 


In [ ]:
import transformers
import pandas as pd
import numpy as np
import transformers
from sklearn import model_selection
from tqdm import trange
import torch

EPOCH = 5
MAX_LEN = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_BASE_PATH = "roberta-base"
HIDDEN_SIZE = 512
NUMBER_OF_LABEL = 30
HIDDEN_DROPOUT_PROB = 0.3
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
TOKENIZER = transformers.RobertaTokenizer.from_pretrained(MODEL_BASE_PATH,do_lower_case = True)

In [ ]:

class GoogleQnADataset(torch.nn.Module):
    def __init__(self,qTitle,qBody,answer,targets,TOKENIZER,MAX_LEN):
      super(GoogleQnADataset,self).__init__()
      self.qTitle = qTitle
      self.qBody = qBody
      self.answer = answer
      self.targets = targets
      self.tokenizer = TOKENIZER
      self.max_len = MAX_LEN

    def __len__(self):
        return len(self.qTitle)

    def __getitem__(self,item):
        question_title = str(self.qTitle[item])
        question_body = str(self.qBody[item])
        answer = str(self.answer[item])

        # [CLS] [Q-TITLE] [Q-BODY] [SEP] [ANSWER] [SEP]

        inputs = self.tokenizer.encode_plus(
            question_title + " " + question_body,
            answer,
            add_special_tokens = True,
            max_length = self.max_len,
            truncation = True,
            padding = True,
            stride = 16
            )

        ids = inputs["input_ids"]
        masks = inputs["attention_mask"]


        padding_length = self.max_len - len(ids)
        ids = ids + ([0] * padding_length)
        masks = masks + ([0] * padding_length)

        return {
            "ids" : torch.tensor(ids,dtype = torch.long),
            "masks" : torch.tensor(masks,dtype = torch.long),
            "targets" : torch.tensor(self.targets[item, :],dtype = torch.float)
        }

In [ ]:
class ROBERTAModel(torch.nn.Module):
    def __init__(self,conf):
        super(ROBERTAModel,self).__init__()
        self.conf = conf
        self.roberta = transformers.RobertaModel(self.conf,add_pooling_layer = False)
        self.dense = torch.nn.Linear(768,768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768,30)

    def forward(self,input_ids,attention_mask):
        output,_ = self.roberta(input_ids,attention_mask,return_dict = False)
        output = output[:,0,:]
        output = self.dropout(output)
        output = self.dense(output)
        output = torch.tanh(output)
        output = self.dropout(output)
        output = self.classifier(output)
        return output


In [ ]:
def Loss(output,target):
    return torch.nn.BCEWithLogitsLoss()(output,target)
    


def Train(DataLoader,Model,Optimizer,Device,Scheduler = None):
    Model.train()

    for index,batch in enumerate(DataLoader):
        ids = batch["ids"]
        masks = batch["masks"]
        target = batch["targets"]

        ids = ids.to(Device,dtype = torch.long)
        masks = masks.to(Device,dtype = torch.long)
        target = target.to(Device,dtype = torch.float)

        Optimizer.zero_grad()

        output = Model(
            input_ids = ids,
            attention_mask = masks
        )
        loss = Loss(output,target)
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(Model.parameters(), 1.0)

        Optimizer.step()

        if Scheduler is not None:
            Scheduler.step()

        if index % 10 == 0:
            print(f"Index {index} >>>====================>>> Train Loss {loss}")



def Eval(DataLoader,Model,Device):
    Model.eval()
    final_outputs = []
    final_targets = []

    for index,batch in enumerate(DataLoader):

        ids = batch["ids"]
        masks = batch["masks"]
        target = batch["targets"]


        ids = ids.to(Device,dtype = torch.long)
        masks = masks.to(Device,dtype = torch.long)
        target = target.to(Device,dtype = torch.float)


        output = Model(
            input_ids = ids,
            attention_mask = masks
        )

        loss = Loss(output,target)

        final_targets.extend(target.cpu().detach().numpy().tolist())
        final_outputs.extend(torch.sigmoid(output).cpu().detach().numpy().tolist())

        if index % 10 == 0:
            print(f"Index : {index} >>>====================>>> Valid Loss : {loss}")
    
    return loss,np.vstack(final_targets), np.vstack(final_outputs)

        

In [ ]:
def optimizer_params(Model):
    param_optimizer = list(Model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    return optimizer_parameters   


In [ ]:

def train():
    # Loading the Data

    df1 = pd.read_csv("/content/drive/MyDrive/Neuron/QnA/train.csv")
    df2 = pd.read_csv("/content/drive/MyDrive/Neuron/QnA/sample_submission.csv")
    # defining the target columns
    target_cols = df2.columns[1:]

    # Spliting the data into Train and Validation

    df_train ,df_valid = model_selection.train_test_split(
        df1,
        test_size = 0.2,
        random_state = 2021,
        )


    # Creating Training and Validation Dataset

    Train_Dataset = GoogleQnADataset(
        qTitle = df_train.question_title.values,
        qBody = df_train.question_body.values,
        answer = df_train.answer.values,
        targets = df_train[target_cols].values,
        TOKENIZER = TOKENIZER,
        MAX_LEN = 128)


    Valid_Dataset = GoogleQnADataset(
        qTitle = df_valid.question_title.values,
        qBody = df_valid.question_body.values,
        answer = df_valid.answer.values,
        targets = df_valid[target_cols].values,
        TOKENIZER = TOKENIZER,
        MAX_LEN = 128)

    # Initilization of Train DataLoader and Validation DataLoader

    Train_DataLoader = torch.utils.data.DataLoader(
        Train_Dataset,
        batch_size =  TRAIN_BATCH_SIZE,
        sampler = torch.utils.data.RandomSampler(Train_Dataset)
    )

    Valid_DataLoader = torch.utils.data.DataLoader(
        Valid_Dataset,
        batch_size = VALID_BATCH_SIZE,
        sampler = torch.utils.data.SequentialSampler(Valid_Dataset)
    )

    # Initilizing the Model
    config = transformers.RobertaConfig.from_pretrained(MODEL_BASE_PATH)
    model = ROBERTAModel(
        conf = config
        )
    model.to(DEVICE)
    
    # Initilizing the optimizer 

    optimizer_grouped_parameters = optimizer_params(Model = model)

    optimizer = transformers.AdamW(
        optimizer_grouped_parameters, 
        lr = 3e-5,
        correct_bias = True)

    # Initilizing the Scheduler

    total_steps = int(len(df1) / TRAIN_BATCH_SIZE * EPOCH)

    scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


    for epoch in trange(EPOCH, desc = "EPOCH"):
        Train(
            DataLoader = Train_DataLoader,
            Model = model, 
            Optimizer = optimizer,
            Device=DEVICE,
            Scheduler=scheduler
        )

        valid_loss,targets,output = Eval(
            DataLoader = Valid_DataLoader,
            Model=model,
            Device = DEVICE
        )

    best_loss = np.inf
    if valid_loss < best_loss:
        torch.save(model.state_dict(),"model.bin")
        valid_loss = best_loss




In [ ]:
train()


EPOCH:   0%|          | 0/5 [00:00<?, ?it/s]

Index 0 >>>====================>>> Train Loss 0.7004292011260986
Index 10 >>>====================>>> Train Loss 0.4854866862297058
Index 20 >>>====================>>> Train Loss 0.4834815263748169
Index 30 >>>====================>>> Train Loss 0.4365706443786621
Index 40 >>>====================>>> Train Loss 0.43352875113487244
Index 50 >>>====================>>> Train Loss 0.4322957694530487
Index 60 >>>====================>>> Train Loss 0.44199302792549133
Index 70 >>>====================>>> Train Loss 0.4446130692958832
Index 80 >>>====================>>> Train Loss 0.4386633038520813
Index 90 >>>====================>>> Train Loss 0.4245954751968384
Index 100 >>>====================>>> Train Loss 0.39313989877700806
Index 110 >>>====================>>> Train Loss 0.4606439173221588
Index 120 >>>====================>>> Train Loss 0.40976691246032715
Index 130 >>>====================>>> Train Loss 0.43538612127304077
Index 140 >>>====================>>> Train Loss 0.4332258403301239
I


EPOCH:  20%|██        | 1/5 [02:16<09:07, 136.79s/it]

Index : 150 >>>====================>>> Valid Loss : 0.39098507165908813
Index 0 >>>====================>>> Train Loss 0.44242027401924133
Index 10 >>>====================>>> Train Loss 0.4347916841506958
Index 20 >>>====================>>> Train Loss 0.419870525598526
Index 30 >>>====================>>> Train Loss 0.4454641044139862
Index 40 >>>====================>>> Train Loss 0.4257974922657013
Index 50 >>>====================>>> Train Loss 0.43188899755477905
Index 60 >>>====================>>> Train Loss 0.4132365584373474
Index 70 >>>====================>>> Train Loss 0.4176747500896454
Index 80 >>>====================>>> Train Loss 0.42237406969070435
Index 90 >>>====================>>> Train Loss 0.47550898790359497
Index 100 >>>====================>>> Train Loss 0.43398982286453247
Index 110 >>>====================>>> Train Loss 0.43590569496154785
Index 120 >>>====================>>> Train Loss 0.42759957909584045
Index 130 >>>====================>>> Train Loss 0.431658416986


EPOCH:  40%|████      | 2/5 [04:34<06:51, 137.16s/it]

...

Index : 150 >>>====================>>> Valid Loss : 0.3893165588378906
Index 0 >>>====================>>> Train Loss 0.41854622960090637
Index 10 >>>====================>>> Train Loss 0.4180167615413666
Index 20 >>>====================>>> Train Loss 0.4159604609012604
Index 30 >>>====================>>> Train Loss 0.42852187156677246
Index 40 >>>====================>>> Train Loss 0.465349018573761
Index 50 >>>====================>>> Train Loss 0.4657309949398041
Index 60 >>>====================>>> Train Loss 0.4169616401195526
Index 70 >>>====================>>> Train Loss 0.44973865151405334
Index 80 >>>====================>>> Train Loss 0.40535038709640503
Index 90 >>>====================>>> Train Loss 0.4192606508731842
Index 100 >>>====================>>> Train Loss 0.4278183877468109
Index 110 >>>====================>>> Train Loss 0.44857650995254517
Index 120 >>>====================>>> Train Loss 0.42834600806236267
Index 130 >>>====================>>> Train Loss 0.41298788785934


EPOCH:  80%|████████  | 4/5 [09:15<02:18, 138.74s/it]

Index : 150 >>>====================>>> Valid Loss : 0.3905406594276428
Index 0 >>>====================>>> Train Loss 0.45993679761886597
Index 10 >>>====================>>> Train Loss 0.41086411476135254
Index 20 >>>====================>>> Train Loss 0.4347713887691498
Index 30 >>>====================>>> Train Loss 0.3980591297149658
Index 40 >>>====================>>> Train Loss 0.41051632165908813
Index 50 >>>====================>>> Train Loss 0.4295395016670227
Index 60 >>>====================>>> Train Loss 0.4164566397666931
Index 70 >>>====================>>> Train Loss 0.4582037627696991
Index 80 >>>====================>>> Train Loss 0.40443238615989685
Index 90 >>>====================>>> Train Loss 0.4225633442401886
Index 100 >>>====================>>> Train Loss 0.4135039150714874
Index 110 >>>====================>>> Train Loss 0.43797767162323
Index 120 >>>====================>>> Train Loss 0.42088985443115234
Index 130 >>>====================>>> Train Loss 0.3959223628044128


EPOCH: 100%|██████████| 5/5 [11:35<00:00, 139.15s/it]

Index : 150 >>>====================>>> Valid Loss : 0.3812680244445801
